<a href="https://colab.research.google.com/github/Decoding-Data-Science/zain/blob/main/Langchain_chains_agent_19thapr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Langchain setup

In [ ]:
!pip install openai langchain langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.2 MB/s eta 0:00:00


In [ ]:
#secret Key

import os
from google.colab import userdata

# Retrieve API keys from Colab's secure storage

openai_api_key = userdata.get("openai")

# Set them as environment variables

if openai_api_key:
    os.environ["OPENAI_API_KEY"] = openai_api_key

## Chains

In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

llm = OpenAI(temperature = 0.9)

#1st chain name_chain
prompt_template_name= PromptTemplate(
    input_variables = ['cuisine'],
    template = "i want to open a restaraunt for  {cuisine} food. Suggest a fancy name for this"
                                         )
name_chain=LLMChain(llm=llm,prompt=prompt_template_name,
output_key="restaraunt_name")

#2nd chain food_items_chain
llm = OpenAI(temperature = 0.9)
prompt_template_items = PromptTemplate(input_variables=['restaraunt_name'],
                                      template="Suggest me some menu items for {restaraunt_name}."
                                       )

food_items_chain=LLMChain(llm=llm,prompt=prompt_template_items,output_key="menu_items")



from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables =['cuisine',],
    output_variables = ['restaraunt_name','menu_items']
)

chain({"cuisine": "Arabic"})

In [ ]:
chain({"cuisine": "Arabic"})

<ipython-input-5-267ddec4af91>:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain({"cuisine": "Arabic"})


{'cuisine': 'Arabic',
 'restaraunt_name': '\n\n"Al Waha Palace" ',
 'menu_items': "\n\n1. Lamb Kebabs: Tender pieces of marinated lamb grilled to perfection and served with a side of rice and vegetables.\n\n2. Chicken Shawarma: Thinly sliced marinated chicken wrapped in a warm pita with pickled vegetables and garlic tahini sauce.\n\n3. Falafel Platter: Traditional deep-fried chickpea and herb patties served with hummus, tabbouleh, and pita bread.\n\n4. Mansaf: Jordanian national dish of slow-cooked lamb served over rice with a rich yogurt sauce and topped with almonds and pine nuts.\n\n5. Shish Tawouk: Grilled chicken skewers marinated in a tangy blend of spices and served with garlic aioli and grilled vegetables.\n\n6. Kofta Kebabs: Ground beef and lamb mixed with herbs and spices, grilled on skewers and served with rice and a tomato-cucumber salad.\n\n7. Mixed Grill Platter: A selection of assorted grilled meats including lamb, chicken, and beef, served with rice, grilled vegetables,

In [ ]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 110.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.6 MB/s eta 0:00:00


In [ ]:
import gradio as gr
from langchain.chains import LLMChain, SequentialChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

# Define the LLM
llm = OpenAI(temperature=0.9)

# Chain 1: Generate restaurant name
prompt_template_name = PromptTemplate(
    input_variables=["cuisine"],
    template="I want to open a restaurant for {cuisine} food. Suggest a fancy name for this."
)
name_chain = LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaraunt_name")

# Chain 2: Generate structured menu
prompt_template_items = PromptTemplate(
    input_variables=["restaraunt_name"],
    template="""
I have opened a new restaurant named "{restaraunt_name}".
Can you suggest a comprehensive menu that includes:

- Starters
- Main Course
- Desserts
- Drinks

Please keep the style creative and aligned with the theme and cuisine suggested by the restaurant's name. Format it clearly in sections.
"""
)
food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")

# Combine the chains
chain = SequentialChain(
    chains=[name_chain, food_items_chain],
    input_variables=["cuisine"],
    output_variables=["restaraunt_name", "menu_items"],
    verbose=True
)

# Define the Gradio function
def generate_restaurant(cuisine):
    result = chain({"cuisine": cuisine})
    return result['restaraunt_name'], result['menu_items']

# Build the UI
iface = gr.Interface(
    fn=generate_restaurant,
    inputs=gr.Textbox(label="Enter Cuisine Type"),
    outputs=[
        gr.Textbox(label="Restaurant Name"),
        gr.Textbox(label="Structured Menu")
    ],
    title="🍽️ Restaurant Business Generator",
    description="Get a fancy restaurant name and a comprehensive menu idea with sections like Starters, Main Course, Desserts, and Drinks!"
)

iface.launch()


ModuleNotFoundError: No module named 'gradio'

## Agents

In [ ]:
# make sure yoy have installed this package: pip install google-search-results
# from secret_key import serpapi_key
# os.environ['SERPAPI_API_KEY'] = serpapi_key
#get your free serp api
os.environ['SERPAPI_API_KEY'] = "722250fb6e1ce4c446e0d12ee5d7c6df99bb5320bdc5988ae20823d0f2e8125c"

In [ ]:
pip install google-search-results

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32009 sha256=e74cca860470bc413445d32108f8315c4dde7a86e7c1c21dac6e19e2887a7314
  Stored in directory: /root/.cache/pip/wheels/6e/42/3e/aeb691b02cb7175ec70e2da04b5658d4739d2b41e5f73cd06f
Successfully built google-search-results


#### serpapi and llm-math tool

In [ ]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["serpapi","wikipedia", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Let's test it out!
agent.run("What was the GDP of US in 2022 plus 5?")

<ipython-input-21-5a97f7cf6115>:10: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
<ipython-input-21-5a97f7cf6115>:13: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent.run("What was the GDP of US in 2022 plu



> Entering new AgentExecutor chain...
 I should use the Calculator tool to solve this math problem.
Action: Calculator
Action Input: 2022 + 5
Observation: Answer: 2027
Thought: Now I need to search for the GDP of the US in 2027.
Action: Search
Action Input: "GDP of US in 2027"

ValueError: Got error from SerpAPI: Your account has run out of searches.

#### Wikipedia and llm-math tool

In [ ]:
pip install wikipedia

In [ ]:
# install this package: pip install wikipedia

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Let's test it out!
agent.run("When was Elon musk born? What is his surrent age")



> Entering new AgentExecutor chain...
 I should use Wikipedia to find information about Elon Musk
Action: wikipedia
Action Input: Elon Musk
Observation: Page: Elon Musk
Summary: Elon Reeve Musk (; born June 28, 1971) is a businessman and conservative political figure known for his key roles in the automotive company Tesla, Inc. and the space company SpaceX. He is also known for his ownership of X Corp. (the company that operates the social media platform X, formerly Twitter), and his role in the founding of the Boring Company, xAI, Neuralink, and OpenAI. Musk is the wealthiest individual in the world; as of January 2025, Forbes estimates his net worth to be US$426 billion.
A member of the wealthy South African Musk family, Musk was born in Pretoria and briefly attended the University of Pretoria. At the age of 18 he immigrated to Canada, acquiring its citizenship through his Canadian-born mother, Maye. Two years later, he matriculated at Queen's University in Canada. Musk later trans

'Elon Musk is currently 54 years old.'